In [ ]:
!pip install opencv-python-headless numpy

In [ ]:
# Create a directory for YOLO files
!mkdir yolo
%cd yolo

# Download YOLOv4 configuration and weights
!wget https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov4.cfg
!wget https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/coco.names
!wget https://pjreddie.com/media/files/yolov4.weights

# Go back to the root directory
%cd ..


In [ ]:
!pip install --upgrade opencv-python-headless numpy

import cv2
import numpy as np

# Re-download the weights file using curl to ensure integrity
!curl -L https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights -o yolo/yolov4.weights

# Load class labels
with open('yolo/coco.names', 'r') as f:
    classes = f.read().strip().split('\n')

# Load YOLO model, ensuring correct paths and disabling CUDA for now
net = cv2.dnn.readNet('yolo/yolov4.weights', 'yolo/yolov4.cfg')
# Commenting out CUDA settings to troubleshoot
# net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
# net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

# Get output layer names
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]

In [ ]:
from google.colab import files
uploaded = files.upload()  # then choose your video file


In [ ]:
from google.colab.patches import cv2_imshow

def detect_and_count_video(video_path, frame_limit=50):
    # Open video file instead of webcam
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video file.")
        return

    frame_count = frame_limit

    while frame_count > 0:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame or end of video reached.")
            break

        height, width = frame.shape[:2]

        # Prepare image for YOLO
        blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), (0, 0, 0), swapRB=True, crop=False)
        net.setInput(blob)
        outputs = net.forward(output_layers)

        boxes = []
        confidences = []
        class_ids = []

        for output in outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]

                if confidence > 0.5:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
        object_count = {}

        if len(indexes) > 0:
            for i in indexes.flatten():
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                confidence = confidences[i]
                color = (0, 255, 0)

                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                cv2.putText(frame, f'{label} {int(confidence * 100)}%', (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                if label in object_count:
                    object_count[label] += 1
                else:
                    object_count[label] = 1

        # Display object count on the frame
        y_offset = 30
        for label, count in object_count.items():
            cv2.putText(frame, f'{label}: {count}', (10, y_offset),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            y_offset += 30

        # Display frame in Colab
        # Convert the frame to uint8 before displaying
        frame = frame.astype(np.uint8)  # Ensure frame data type is uint8
        cv2_imshow(frame)

        frame_count -= 1

    cap.release()
    cv2.destroyAllWindows()

# Replace 'video.mp4' with the name of your uploaded video file.
detect_and_count_video('your_video.mp4', frame_limit=50)